In [2524]:
import random
import math
import time

In [2525]:
start = time.time()
print(start)

1608067754.832611


In [2526]:
myfile = open("./Input/customSokoban_13x13_FurtherBox_00c.txt")

lines = []
for eachline in myfile:
    lines.append(eachline)
myfile.close()

In [2527]:
# Input Line0
sizeH = int(lines[0].split()[0])
sizeV = int(lines[0].split()[1])

In [2528]:
# Loading data into a set
# Arrays are in (y,x), with top row as 1, and leftmost column as 1
def createCoordinatesSet(inputArray):
    coordinates = set([])
    for i,j in zip(inputArray[0::2], inputArray[1::2]):
        coordinates.add((int(i), int(j)))
    return coordinates

In [2529]:
# Input Line1
line1Array = lines[1].split()
nWallSquares = int(line1Array[0])
wallCoordinates = createCoordinatesSet(line1Array[1:])

if (len(wallCoordinates)!=nWallSquares):
    print("Wall square does not match, check input")

In [2530]:
# Input Line2
line2Array = lines[2].split()
nBoxes = int(line2Array[0])
boxCoordinates = createCoordinatesSet(line2Array[1:])

if (len(boxCoordinates)!=nBoxes):
    print("Boxes does not match, check input")

In [2531]:
# Input Line3
line3Array = lines[3].split()
nStorageLocations = int(line3Array[0])
storageCoordinates = createCoordinatesSet(line3Array[1:])

if (len(storageCoordinates)!=nStorageLocations):
    print("Wall square does not match, check input")

In [2532]:
# Input Line4
line4Array = lines[4].split()
initialLocation = (int(line4Array[0]), int(line4Array[1]))

In [2533]:
#constants
actions = ['U', 'D', 'L', 'R']
discount = 1.0
learningRate = 0.7

In [2534]:
class SokobanState:
    def __init__(self, wallCoordinates, boxCoordinates, storageCoordinates, location):
        self.WallCoordinates = frozenset(wallCoordinates.copy())
        self.BoxCoordinates = frozenset(boxCoordinates.copy())
        self.StorageCoordinates = frozenset(storageCoordinates.copy())
        self.Location = location
    
    def __hash__(self):
        return hash((self.BoxCoordinates, self.Location))

    def __eq__(self, other):
        return (self.BoxCoordinates, self.Location) == (other.BoxCoordinates, other.Location)
    
    def minStepsBetweenCoordinates(self, coordinates1, coordinates2):
        return abs(coordinates1[0]-coordinates2[0]) + abs(coordinates1[1]-coordinates2[1])
    
    def totalBoxToClosestStorageSteps(self):
        result = 0
        for box in self.BoxCoordinates.difference(self.StorageCoordinates):
            minFound = float("inf")
            for storage in self.StorageCoordinates.difference(self.BoxCoordinates):
                steps = self.minStepsBetweenCoordinates(box, storage)
                if steps < minFound:
                    minFound = steps
            result += minFound
        return result
    
    def agentToClosestBoxSteps(self):
        minFound = float("inf")
        for box in self.BoxCoordinates.difference(self.StorageCoordinates):
            steps = self.minStepsBetweenCoordinates(self.Location, box)
            if steps < minFound:
                minFound = steps
        return minFound
    
    def remainingBoxes(self):
        count = 0
        for box in self.BoxCoordinates.difference(self.StorageCoordinates):
            count += 1
        return count
    
    def isTerminal(self):
        return all(map(lambda BoxCoor: BoxCoor in self.StorageCoordinates, self.BoxCoordinates))
    
    def boxStuckAtCorner(self, box):
        if box in self.StorageCoordinates:
            return False
        coordinateDifference = [(-1,-1), (-1,+1), (+1, -1), (+1,+1)]
        for difference in coordinateDifference:
            if (box[0]+difference[0], box[1]) in self.WallCoordinates and (box[0], box[1]+difference[1]) in self.WallCoordinates:
                return True
        return False
    
    def noSolution(self):
        return any(map(lambda BoxCoor: self.boxStuckAtCorner(BoxCoor), self.BoxCoordinates))
    
    def getTargetLocations(self, action):
        currentLocation = self.Location
        targetLocation = currentLocation
        targetNextLocation = (currentLocation[0]+1, currentLocation[1])
        if action == 'U':
            targetLocation = (currentLocation[0]-1, currentLocation[1])
            targetNextLocation = (currentLocation[0]-2, currentLocation[1])
        elif action == 'D':
            targetLocation = (currentLocation[0]+1, currentLocation[1])
            targetNextLocation = (currentLocation[0]+2, currentLocation[1])
        elif action == 'L':
            targetLocation = (currentLocation[0], currentLocation[1]-1)
            targetNextLocation = (currentLocation[0], currentLocation[1]-2)
        elif action == 'R':
            targetLocation = (currentLocation[0], currentLocation[1]+1)
            targetNextLocation = (currentLocation[0], currentLocation[1]+2)
        return (targetLocation, targetNextLocation)
    
    def isInvalidAction(self, action):
        targetLocation, targetNextLocation = self.getTargetLocations(action)
        return (targetLocation in self.WallCoordinates) or (targetLocation in self.BoxCoordinates and targetNextLocation in self.BoxCoordinates) or (targetLocation in self.BoxCoordinates and targetNextLocation in self.WallCoordinates)
    
    # Avoid agent stuck at local maximum by staying at the same location, only allow movable actions
    def getPossibleActions(self):
        return [action for action in actions if not self.isInvalidAction(action)]

In [2535]:
class Sokoban:
    qFunction = {}
    
    def __init__(self, sizeH, sizeV, wallCoordinates, boxCoordinates, storageCoordinates, location):
        self.SizeH = sizeH
        self.SizeV = sizeV
        self.currentState = SokobanState(wallCoordinates.copy(), boxCoordinates.copy(), storageCoordinates.copy(), location)

    def resetQFunction(self):
        self.__class__.qFunction.clear()
        
    def getQValue(self, state, action):
        if state.isTerminal():
            return 10000000
        if state.noSolution():
            return -10000000
        return self.__class__.qFunction.get((state, action), 0)
    
    def setQValue(self, state, action, newValue):
        self.__class__.qFunction[(state, action)] = newValue
    
    def getCurrentStateActions(self):
        return self.currentState.getPossibleActions()
    
    def getMaxQValue(self, state, possibleActions):
        return max(map(lambda action: self.getQValue(state, action), possibleActions))
    
    def getBestAction(self, state):
        possibleActions = state.getPossibleActions()
        maxQValue = self.getMaxQValue(state, possibleActions)
        for action in possibleActions:
            if self.getQValue(state, action) == maxQValue:
                return action
    
    # create S'
    def createStateForAction(self, state, action):
        targetLocation, targetNextLocation = state.getTargetLocations(action)
        newLocation = targetLocation
        # map frozenset back to a mutable set
        newBoxCoordinates = set(state.BoxCoordinates.copy())
        if targetLocation in newBoxCoordinates:
            newBoxCoordinates.remove(targetLocation)
            newBoxCoordinates.add(targetNextLocation)
        return SokobanState(state.WallCoordinates, newBoxCoordinates, state.StorageCoordinates, newLocation)
        
    # Take action A, observe R and S'
    # Update Q value
    # Update S to S'
    def takeAction(self, action):
        newState = self.createStateForAction(self.currentState, action)
        
        # Reward R
        R = -1
        # Reward for moving box closer
        stepDifference = newState.totalBoxToClosestStorageSteps() - self.currentState.totalBoxToClosestStorageSteps()
        if stepDifference < 0:
            R += 3
        elif stepDifference == 0:
            R += -3
        
        # Reward for agent getting closer to a box
        distanceToClosestBoxDifference = newState.agentToClosestBoxSteps() - self.currentState.agentToClosestBoxSteps()
        if distanceToClosestBoxDifference < 0:
            R += 1
        elif distanceToClosestBoxDifference > 0:
            R += -1
        
        # Reward for moving box to Storage
        remainingBoxesDifference = newState.remainingBoxes() - self.currentState.remainingBoxes()
        if remainingBoxesDifference < 0:
            R += 15
        elif remainingBoxesDifference > 0:
            R += -10
        
        currentQValue = self.getQValue(self.currentState, action)
        newQValue = currentQValue + learningRate*(R+discount*self.getMaxQValue(newState, newState.getPossibleActions())-currentQValue)
        self.setQValue(self.currentState, action, newQValue)
        self.currentState = newState
    
    def getPath(self, maxSteps, startState):
        path = ''
        prevState = startState
        tmpState = startState
        for i in range(maxSteps):
            if tmpState.isTerminal():
                print("Reached terminal state")
                break
            
            # May need this part to avoid stuck at local maximum
#             bestNextState = None
#             bestAction = None
#             bestQ = -float("inf")
#             for action in tmpState.getPossibleActions():
#                 checkState = self.createStateForAction(tmpState, action)
#                 qValue = self.getQValue(tmpState, action)
#                 if qValue >= bestQ and checkState != prevState:
#                     bestState = checkState
#                     bestAction = action
#                     bestQ = qValue
#             print(tmpState.getPossibleActions())
#             print(list(map(lambda action: self.getQValue(tmpState, action), tmpState.getPossibleActions())))
#             print(bestAction)
#             print(self.getBestAction(tmpState))
#             print(tmpState.Location)
#             path += bestAction
#             prevState = tmpState
#             tmpState = bestState
                    
            bestAction = self.getBestAction(tmpState)
            newState = self.createStateForAction(tmpState, bestAction)
            path += bestAction
            prevState = tmpState
            tmpState = newState
           
        return path

In [2536]:
sokoban = Sokoban(sizeH, sizeV, wallCoordinates, boxCoordinates, storageCoordinates, initialLocation)
sokoban.resetQFunction()

In [2537]:
random.seed(0)
def createRandomStartLocation():
    randomLocation = (1,1)
    while randomLocation in wallCoordinates or randomLocation in boxCoordinates or randomLocation in storageCoordinates:
        randomLocation = (random.randint(1, sizeH), random.randint(1, sizeV))
    return randomLocation

In [2538]:
def createAndRunEpisode(startLocation, maxSteps, epsilon):
    # Return True if this episode reached terminal state
    sokoban = Sokoban(sizeH, sizeV, wallCoordinates, boxCoordinates, storageCoordinates, startLocation)
    for j in range(maxSteps):
        if random.random() < epsilon:
            action = random.choice(sokoban.currentState.getPossibleActions())
        else:
            action = sokoban.getBestAction(sokoban.currentState)
        sokoban.takeAction(action)
        if sokoban.currentState.isTerminal():
            print("Stpes used to reach terminal: ", j+1)
            return True
#         if sokoban.currentState.noSolution():
#             return False
    return False

In [2539]:
episodes = sizeH*sizeV*nBoxes*4*100
maxSteps = sizeH*sizeV*nBoxes
terminalCount = 0
epsilon = 0.2
print('Total episodes: ', episodes)
# Q-learning
for i in range(episodes):
    episodeReachedTerminal = createAndRunEpisode(initialLocation, maxSteps, epsilon)
    if episodeReachedTerminal:
        terminalCount += 1
    if i%1000 == 0:
        print("Total Completed Episodes: ", i)
        print("Episodes reached terminal state in the last 1000 Episodes: ", terminalCount)
        tmpSokoban = Sokoban(sizeH, sizeV, wallCoordinates, boxCoordinates, storageCoordinates, initialLocation)
        currentBestPath = tmpSokoban.getPath(maxSteps, tmpSokoban.currentState)
        print(len(currentBestPath), " ", currentBestPath)
        # if we have a good enough policy, we should have a high chance of reaching the terminal state, so we can terminate
        if terminalCount > 500:
            print("Early terminate")
            break
        timeElapsed = time.time() - start
        print('Time elapsed: ', timeElapsed)
        if timeElapsed > 3600:
            print('Timeout')
            break
        terminalCount = 0

Total episodes:  202800
Total Completed Episodes:  0
Episodes reached terminal state in the last 1000 Episodes:  0
507   ULRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLR
Time elapsed:  0.14612793922424316
Stpes used to reach terminal:  307
Stpes used to reach terminal:  129
Total Completed Episodes:  1000
Episodes reached terminal state in the last 1000 Episodes:  2
507   UUUUUULDDDDRDLLLULLDRRRRDRUUUUUULUURULLLULDDDLDRUULRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLRLR

Total Completed Episodes:  10000
Episodes reached terminal state in the last 1000 Episodes:  9
507   UUUUUULUURULLLLLLULLDRRRRRRRURDDDDDDLDDRUUUUUDDDDDDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUD
Time elapsed:  281.9654767513275
Stpes used to reach terminal:  474
Stpes used to reach terminal:  140
Stpes used to reach terminal:  254
Stpes used to reach terminal:  115
Stpes used to reach terminal:  449
Stpes used to reach terminal:  451
Stpes used to reach terminal:  432
Total Completed Episodes:  11000
Episodes reached terminal state in the last 1000 Episodes:  7
507   UUUUUULUURUL

Stpes used to reach terminal:  422
Stpes used to reach terminal:  282
Stpes used to reach terminal:  279
Stpes used to reach terminal:  183
Stpes used to reach terminal:  95
Stpes used to reach terminal:  339
Stpes used to reach terminal:  105
Stpes used to reach terminal:  101
Stpes used to reach terminal:  174
Stpes used to reach terminal:  389
Stpes used to reach terminal:  115
Stpes used to reach terminal:  485
Stpes used to reach terminal:  93
Stpes used to reach terminal:  117
Stpes used to reach terminal:  300
Stpes used to reach terminal:  97
Stpes used to reach terminal:  101
Stpes used to reach terminal:  101
Stpes used to reach terminal:  129
Stpes used to reach terminal:  97
Stpes used to reach terminal:  133
Stpes used to reach terminal:  117
Total Completed Episodes:  19000
Episodes reached terminal state in the last 1000 Episodes:  22
507   UUUUUULDUUURULLLLLLULLDRRRRRRRURDDDDDLDDDDRDLLLLLUUUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDUDU

Stpes used to reach terminal:  87
Stpes used to reach terminal:  97
Stpes used to reach terminal:  91
Stpes used to reach terminal:  158
Stpes used to reach terminal:  85
Stpes used to reach terminal:  89
Stpes used to reach terminal:  83
Stpes used to reach terminal:  109
Stpes used to reach terminal:  95
Stpes used to reach terminal:  97
Stpes used to reach terminal:  99
Stpes used to reach terminal:  97
Stpes used to reach terminal:  95
Stpes used to reach terminal:  97
Stpes used to reach terminal:  93
Stpes used to reach terminal:  127
Stpes used to reach terminal:  113
Stpes used to reach terminal:  99
Stpes used to reach terminal:  111
Stpes used to reach terminal:  109
Stpes used to reach terminal:  99
Stpes used to reach terminal:  95
Stpes used to reach terminal:  103
Stpes used to reach terminal:  99
Stpes used to reach terminal:  87
Stpes used to reach terminal:  93
Stpes used to reach terminal:  91
Stpes used to reach terminal:  89
Stpes used to reach terminal:  119
Stpes 

Stpes used to reach terminal:  101
Stpes used to reach terminal:  105
Stpes used to reach terminal:  95
Stpes used to reach terminal:  91
Stpes used to reach terminal:  115
Stpes used to reach terminal:  97
Stpes used to reach terminal:  95
Stpes used to reach terminal:  103
Stpes used to reach terminal:  121
Stpes used to reach terminal:  95
Stpes used to reach terminal:  107
Stpes used to reach terminal:  93
Stpes used to reach terminal:  93
Stpes used to reach terminal:  115
Stpes used to reach terminal:  103
Stpes used to reach terminal:  91
Stpes used to reach terminal:  91
Stpes used to reach terminal:  115
Stpes used to reach terminal:  91
Stpes used to reach terminal:  95
Stpes used to reach terminal:  107
Stpes used to reach terminal:  107
Stpes used to reach terminal:  105
Stpes used to reach terminal:  99
Stpes used to reach terminal:  97
Stpes used to reach terminal:  91
Stpes used to reach terminal:  135
Stpes used to reach terminal:  95
Stpes used to reach terminal:  99
S

Stpes used to reach terminal:  101
Stpes used to reach terminal:  91
Stpes used to reach terminal:  101
Stpes used to reach terminal:  97
Stpes used to reach terminal:  99
Stpes used to reach terminal:  87
Stpes used to reach terminal:  101
Stpes used to reach terminal:  87
Stpes used to reach terminal:  95
Stpes used to reach terminal:  119
Stpes used to reach terminal:  99
Stpes used to reach terminal:  93
Stpes used to reach terminal:  103
Stpes used to reach terminal:  107
Stpes used to reach terminal:  93
Stpes used to reach terminal:  103
Stpes used to reach terminal:  91
Stpes used to reach terminal:  89
Stpes used to reach terminal:  95
Stpes used to reach terminal:  99
Stpes used to reach terminal:  107
Stpes used to reach terminal:  87
Stpes used to reach terminal:  95
Stpes used to reach terminal:  97
Stpes used to reach terminal:  97
Stpes used to reach terminal:  103
Stpes used to reach terminal:  105
Stpes used to reach terminal:  105
Stpes used to reach terminal:  89
Stp

In [2540]:
sokoban = Sokoban(sizeH, sizeV, wallCoordinates, boxCoordinates, storageCoordinates, initialLocation)
path = sokoban.getPath(maxSteps, sokoban.currentState)
print(len(path), " ", path)

Reached terminal state
75   UUUUUULDDDDRDLDLLURRDRUUUUUULUURULLLULDDDLDRULUULLULDDDDDDDDRDLLDLUUUUUUUUU


In [2541]:
end = time.time()
print('Execution time: ', end - start)

Execution time:  572.8391230106354
